In [1]:
import pandas as pd
import numpy as np

In [2]:
from catboost import CatBoostRegressor

In [3]:
loc_lbl = r"D:\TP2_Machine_Learning\Labels"

In [4]:
loc_ftr = r"D:\TP2_Machine_Learning\Features\ftr_ins"

In [5]:
ins_lb = pd.read_csv( loc_lbl + "\\ins_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [6]:
rh_18_20 = pd.read_csv( "D:\TP2_Machine_Learning\Training Sets\ins_18_20.csv" ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( "D:\TP2_Machine_Learning\Training Sets\ins_21_23.csv" ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [7]:
#rh_18_20 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\ins_18_20.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_18_20 = rh_18_20.drop_duplicates(subset=["ref_hash"])
#rh_21_23 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\ins_21_23.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_21_23 = rh_21_23.drop_duplicates(subset=["ref_hash"])

In [8]:
#rh_18_20 = rh_18_20.loc[:,['ref_hash']]
#rh_21_23 = rh_21_23.loc[:,['ref_hash']]
#rh_18_20.to_csv(r"D:\TP2_Machine_Learning\Training Sets\rh_18_20_sc.csv", index = False)
#rh_21_23.to_csv(r"D:\TP2_Machine_Learning\Training Sets\rh_21_23_sc.csv", index = False)

In [9]:
dt_18_20 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
sc_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["21_23_sc"] ] #SC del (21-23)

dt_21_23 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
sc_24_26 = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["24_26_sc"] ] #SC del (24-26)

In [12]:
#sc_24_26

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [13]:
set_trn = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash","21_23_sc"] ]
set_tst = ins_lb.loc[ ins_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash", "24_26_sc"] ]

t_lim = 259200.00
max_t = 200

set_trn_loc = pd.concat( [ set_trn.loc[ set_trn["21_23_sc"] < t_lim ], set_trn.loc[ set_trn["21_23_sc"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 
set_tst_loc = pd.concat( [ set_tst.loc[ set_tst["24_26_sc"] < t_lim ], set_tst.loc[ set_tst["24_26_sc"] == t_lim ].reset_index( drop = True ).loc[: max_t] ], axis = 0 ).sample(frac=1).reset_index(drop=True) 

In [14]:
X_loc = set_trn_loc.loc[:,['ref_hash']] #Datos train
Y_loc = set_trn_loc.loc[:,['21_23_sc']] #Label train

Z_loc = set_tst_loc.loc[:,['ref_hash']] #Datos test
W_loc = set_tst_loc.loc[:,['24_26_sc']] #Label test

In [176]:
X_var = X_loc
Y_var = Y_loc
Z_var = Z_loc
W_var = W_loc

In [177]:
### Hacer features de ultima instalacion/subasta/evento, tiempo de instalacion (21-3, 4-10, 11-15, 16-20) y si hubieron subastas/eventos/clicks a la noche

In [178]:
ftr_01_Z = pd.read_csv( loc_ftr + "\\sin_hour_21_23.csv" ) 
ftr_02_Z = pd.read_csv( loc_ftr + "\\cant_ins_21_23.csv" )
#ftr_03_Z = pd.read_csv( loc_ftr + "\\cins_imp_21_23.csv" )
#ftr_04_Z = pd.read_csv( loc_ftr + "\\cant_evt_21_23.csv" ) <-bad
#ftr_05_Z = pd.read_csv( loc_ftr + "\\cant_auc_21_23.csv" )
#ftr_06_Z = pd.read_csv( loc_ftr + "\\wifi_ins_21_23.csv" )
#ftr_07_Z = pd.read_csv( loc_ftr + "\\main_app_21_23.csv" )
#ftr_08_Z = pd.read_csv( loc_ftr + "\\type_ins_21_23.csv" )
#ftr_09_Z = pd.read_csv( loc_ftr + "\\ref_type_21_23.csv" )
#ftr_10_Z = pd.read_csv( loc_ftr + "\\ip_encod_21_23.csv" )
#ftr_11_Z = pd.read_csv( loc_ftr + "\\cevt_atr_21_23.csv" )
#ftr_12_Z = pd.read_csv( loc_ftr + "\\kind_evt_21_23.csv" )
ftr_13_Z = pd.read_csv( loc_ftr + "\\frst_auc_21_23.csv" )
ftr_14_Z = pd.read_csv( loc_ftr + "\\frst_evt_21_23.csv" )
#ftr_15_Z = pd.read_csv( loc_ftr + "\\hr_f_evt_21_23.csv" )

ftr_16_Z = pd.read_csv( loc_ftr + "\\last_ins_21_23.csv" )
ftr_17_Z = pd.read_csv( loc_ftr + "\\last_auc_21_23.csv" )
ftr_18_Z = pd.read_csv( loc_ftr + "\\last_evt_21_23.csv" )
ftr_19_Z = pd.read_csv( loc_ftr + "\\last_clk_21_23.csv" )
ftr_20_Z = pd.read_csv( loc_ftr + "\\me_mt_1_ins_21_23.csv" )
ftr_21_Z = pd.read_csv( loc_ftr + "\\mt_1_ins_21_23.csv" )

ftr_22_Z = pd.read_csv( loc_ftr + "\\me_ins_night_21_23.csv" )
#ftr_23_Z = pd.read_csv( loc_ftr + "\\ins_night_21_23.csv" )
ftr_24_Z = pd.read_csv( loc_ftr + "\\me_ins_morn_21_23.csv" )
#ftr_25_Z = pd.read_csv( loc_ftr + "\\ins_morn_21_23.csv" )
ftr_26_Z = pd.read_csv( loc_ftr + "\\me_ins_midday_21_23.csv" )
#ftr_27_Z = pd.read_csv( loc_ftr + "\\ins_midday_21_23.csv" )
ftr_28_Z = pd.read_csv( loc_ftr + "\\me_ins_after_21_23.csv" )
#ftr_29_Z = pd.read_csv( loc_ftr + "\\ins_after_21_23.csv" )


In [179]:
Z_var = Z_var.merge( ftr_01_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_02_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_03_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_04_Z, how = "inner", on = "ref_hash" ) <- bad
#Z_var = Z_var.merge( ftr_05_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_06_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_07_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_08_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_09_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_10_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_11_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_12_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_13_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_14_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_15_Z, how = "inner", on = "ref_hash" )

Z_var = Z_var.merge( ftr_16_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_17_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_18_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_19_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_20_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_21_Z, how = "inner", on = "ref_hash" )

Z_var = Z_var.merge( ftr_22_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_23_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_24_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_25_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_26_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_27_Z, how = "inner", on = "ref_hash" )
Z_var = Z_var.merge( ftr_28_Z, how = "inner", on = "ref_hash" )
#Z_var = Z_var.merge( ftr_29_Z, how = "inner", on = "ref_hash" )


# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

In [180]:
#Y = sc_21_23

In [181]:
#X = dt_18_20

### Agregamos los features

In [182]:
ftr_01_X = pd.read_csv( loc_ftr + "\\sin_hour_18_20.csv" )
ftr_02_X = pd.read_csv( loc_ftr + "\\cant_ins_18_20.csv" )
#ftr_03_X = pd.read_csv( loc_ftr + "\\cins_imp_18_20.csv" )
#ftr_04_X = pd.read_csv( loc_ftr + "\\cant_evt_18_20.csv" ) <- bad
#ftr_05_X = pd.read_csv( loc_ftr + "\\cant_auc_18_20.csv" )
#ftr_06_X = pd.read_csv( loc_ftr + "\\wifi_ins_18_20.csv" )
#ftr_07_X = pd.read_csv( loc_ftr + "\\main_app_18_20.csv" )
#ftr_08_X = pd.read_csv( loc_ftr + "\\type_ins_18_20.csv" )
#ftr_09_X = pd.read_csv( loc_ftr + "\\ref_type_18_20.csv" )
#ftr_10_X = pd.read_csv( loc_ftr + "\\ip_encod_18_20.csv" )
#ftr_11_X = pd.read_csv( loc_ftr + "\\cevt_atr_18_20.csv" )
#ftr_12_X = pd.read_csv( loc_ftr + "\\kind_evt_18_20.csv" )
ftr_13_X = pd.read_csv( loc_ftr + "\\frst_auc_18_20.csv" )
ftr_14_X = pd.read_csv( loc_ftr + "\\frst_evt_18_20.csv" )
#ftr_15_X = pd.read_csv( loc_ftr + "\\hr_f_evt_18_20.csv" )

ftr_16_X = pd.read_csv( loc_ftr + "\\last_ins_18_20.csv" )
ftr_17_X = pd.read_csv( loc_ftr + "\\last_auc_18_20.csv" )
ftr_18_X = pd.read_csv( loc_ftr + "\\last_evt_18_20.csv" )
ftr_19_X = pd.read_csv( loc_ftr + "\\last_clk_18_20.csv" )
ftr_20_X = pd.read_csv( loc_ftr + "\\me_mt_1_ins_18_20.csv" )
ftr_21_X = pd.read_csv( loc_ftr + "\\mt_1_ins_18_20.csv" )

ftr_22_X = pd.read_csv( loc_ftr + "\\me_ins_night_18_20.csv" )
#ftr_23_X = pd.read_csv( loc_ftr + "\\ins_night_18_20.csv" )
ftr_24_X = pd.read_csv( loc_ftr + "\\me_ins_morn_18_20.csv" )
#ftr_25_X = pd.read_csv( loc_ftr + "\\ins_morn_18_20.csv" )
ftr_26_X = pd.read_csv( loc_ftr + "\\me_ins_midday_18_20.csv" )
#ftr_27_X = pd.read_csv( loc_ftr + "\\ins_midday_18_20.csv" )
ftr_28_X = pd.read_csv( loc_ftr + "\\me_ins_after_18_20.csv" )
#ftr_29_X = pd.read_csv( loc_ftr + "\\ins_after_18_20.csv" )


In [183]:
X_var = X_var.merge( ftr_01_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_02_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_03_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_04_X, how = "inner", on = "ref_hash" ) <- bad
#X_var = X_var.merge( ftr_05_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_06_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_07_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_08_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_09_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_10_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_11_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_12_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_13_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_14_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_15_X, how = "inner", on = "ref_hash" )

X_var = X_var.merge( ftr_16_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_17_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_18_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_19_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_20_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_21_X, how = "inner", on = "ref_hash" )

X_var = X_var.merge( ftr_22_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_23_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_24_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_25_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_26_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_27_X, how = "inner", on = "ref_hash" )
X_var = X_var.merge( ftr_28_X, how = "inner", on = "ref_hash" )
#X_var = X_var.merge( ftr_29_X, how = "inner", on = "ref_hash" )


## Dropeamos los ref_hash (No estan codificados)

In [184]:
X_var = X_var.drop( "ref_hash", axis = 1 )
Z_var = Z_var.drop( "ref_hash", axis = 1 )

# Prediccion con <span style="color:green"> *CATBOOST*</span> 

### X: Train Data, Y: Train Labels, Z: Test Data, sc_24_26: Test Labels

### Record = iterations=300, learning_rate=0.06, depth=3, RMSE: 25506.739133

In [91]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
mse = make_scorer( mean_squared_error, greater_is_better = False ) 

In [185]:
model = CatBoostRegressor(loss_function='RMSE')
parameters = {'depth'         : [2,3,6],
              'learning_rate' : [0.04, 0.06, 0.1],
              'iterations'    : [20, 50, 200]
                 }
grid = GridSearchCV(estimator=model, scoring=mse ,param_grid = parameters, cv = 4, n_jobs=2)
grid.fit(X_var, Y_var)  

0:	learn: 138456.9180744	total: 10.6ms	remaining: 518ms
1:	learn: 129506.9011194	total: 19.9ms	remaining: 477ms
2:	learn: 121786.6611549	total: 30.9ms	remaining: 484ms
3:	learn: 115143.4106864	total: 46ms	remaining: 529ms
4:	learn: 109459.8126835	total: 56.5ms	remaining: 508ms
5:	learn: 104583.8411013	total: 66.7ms	remaining: 489ms
6:	learn: 100496.0984313	total: 77ms	remaining: 473ms
7:	learn: 97060.7355856	total: 87.8ms	remaining: 461ms
8:	learn: 94176.3939548	total: 99.4ms	remaining: 453ms
9:	learn: 91755.6942576	total: 110ms	remaining: 439ms
10:	learn: 89755.9709516	total: 120ms	remaining: 426ms
11:	learn: 88105.3073372	total: 130ms	remaining: 413ms
12:	learn: 86696.1292110	total: 141ms	remaining: 402ms
13:	learn: 85568.6906816	total: 151ms	remaining: 389ms
14:	learn: 84610.3879704	total: 163ms	remaining: 380ms
15:	learn: 83849.9259179	total: 173ms	remaining: 368ms
16:	learn: 83233.9413361	total: 183ms	remaining: 354ms
17:	learn: 82732.9524772	total: 193ms	remaining: 343ms
18:	lear

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=<catboost.core.CatBoostRegressor object at 0x000000000AC72860>,
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'depth': [2, 3, 6], 'learning_rate': [0.04, 0.06, 0.1], 'iterations': [20, 50, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [186]:
    # Results from Grid Search
    print("\n========================================================")
    print(" Results from Grid Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
    
    print("\n ========================================================")


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 -6481350505.230204

 The best parameters across ALL searched params:
 {'depth': 2, 'iterations': 50, 'learning_rate': 0.1}



In [187]:
preds = grid.predict(Z_var)

### Parametros encontrados con grid-searchCV

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=200,
                          learning_rate=0.06,
                          depth=3,
                          loss_function='RMSE')

# Fit model
model.fit(X_var, Y_var)

preds = model.predict(Z_var)

### Calculamos el RMSE

In [95]:
from sklearn.metrics import mean_squared_error

In [188]:
rmse = np.sqrt( mean_squared_error(W_var, preds) )
print("RMSE: %f" % (rmse) )

RMSE: 81184.420016


In [189]:
record = 81101.00

In [174]:
W_var.head(10)

,24_26_sc
0,229990.266
1,205604.857
2,77252.910
3,87122.160
4,228383.130
5,257903.369
6,190868.366
7,198149.001
8,167000.260
9,95522.944


In [175]:
pd.DataFrame(preds).head(10)

,0
0,124348.696202
1,128601.166884
2,100930.615748
3,126457.570082
4,121662.226277
5,126131.980279
6,122844.721198
7,128185.274896
8,125104.745964
9,122309.411497
